# First step
We use demucs to separate vocals

# Second

We get a rough timestamp(using kotoba-tech/kotoba-whisper-v2.0-faster, 4m processing for a 2h movie) and an accurate transcript without timestamp(using [amine whisper](https://huggingface.co/litagin/anime-whisper))

In [ ]:
import subprocess
from pathlib import Path

import torch
from transformers import pipeline

import os
import openai
import json

In [ ]:

def extract_audio(input_video: str,
                  sample_rate: int = 16000,
                  channels: int = 1,
                  codec: str = "pcm_s16le") -> Path:
    """
    从任意视频文件中提取音频，输出 WAV 并返回输出文件路径。
    文件名：保留原始 stem，后缀改为 .wav
    """
    in_path = Path(input_video)
    stem = in_path.stem
    audio_output = in_path.with_name(f"{stem}.wav")

    cmd = [
        "ffmpeg",
        "-y",  # 如果输出已存在则覆盖
        "-i", str(in_path),
        "-vn",
        "-acodec", codec,
        "-ar", str(sample_rate),
        "-ac", str(channels),
        str(audio_output)
    ]
    subprocess.run(cmd, check=True)
    return audio_output


def separate_vocals(audio_input: Path,
                    stems: str = "vocals",
                    model: str = "htdemucs") -> None:
    """
    对提取后的音频做人声分离，两声道模式下只保留 vocals。
    Demucs 会在当前目录下创建一个 separated/<model>/<stem> 文件夹。
    """
    cmd = [
        "python", "-m", "demucs",
        f"--two-stems={stems}",
        "-n", model,
        str(audio_input)
    ]
    subprocess.run(cmd, check=True)


video_path = Path("D:/Downloads/AAA.mp4")

# 提取音频，自动生成 your_movie.wav
wav_path = extract_audio(video_path,
                            sample_rate=16000,
                            channels=1)

# 基于 your_movie.wav 分离人声
demucs_model = "htdemucs" 
separate_vocals(wav_path,
                stems="vocals",
                model=demucs_model)

# 分离后的人声文件路径
vocals_path = Path(f"separated/{demucs_model}/{wav_path.stem}/vocals.wav")\

print(f"分离后的人声文件路径: {vocals_path}")


In [ ]:
generate_kwargs = {
    "language": "Japanese",
    "no_repeat_ngram_size": 0,
    "repetition_penalty": 1.0,
}
pipe = pipeline(
    "automatic-speech-recognition",
    model="./models/anime",
    device="cuda",

    torch_dtype=torch.float16,
    chunk_length_s=30.0,
    batch_size=64,
)

result = pipe(vocals_path, generate_kwargs=generate_kwargs)

os.makedirs("transcripts", exist_ok=True)
with open(f"transcripts/{wav_path.stem}.txt", "w", encoding="utf-8") as f:
    f.write(result["text"])



out_dir = Path(f"timestamps/{wav_path.stem}")
out_dir.mkdir(exist_ok=True)

cmd = [
    "whisperx", str(vocals_path),
    "--model", "tiny",
    "--device", "cuda",
    "--language", "ja",
    "--vad_method", "silero",
    "--chunk_size", "6",
    "--batch_size", "16",
    "--compute_type", "float16",
    # "--output_format", "json",
    "--highlight_words", "True",
    "--output_dir", str(out_dir)
]
subprocess.run(cmd, check=True)

# Third step
Use openai API to align the trancription to timestamps

In [ ]:
client = openai.OpenAI(api_key=os.environ["OPENAI_API_KEY"]) # Ensure you have set your OpenAI API key in the environment variable or .env file


def load_data(json_path, accurate_text_path):
    """加载JSON时间戳数据和精确的文本文件"""
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            timing_data = json.load(f)
    except FileNotFoundError:
        print(f"错误: JSON文件未找到 at {json_path}")
        return None, None
    except json.JSONDecodeError:
        print(f"错误: JSON文件格式不正确 at {json_path}")
        return None, None

    try:
        with open(accurate_text_path, 'r', encoding='utf-8') as f:
            accurate_text = f.read()
    except FileNotFoundError:
        print(f"错误: 精确文本文件未找到 at {accurate_text_path}")
        return None, None
        
    return timing_data, accurate_text

def format_timing_data_for_prompt(timing_data):
    """将JSON数据格式化为更易于LLM阅读的字符串"""
    formatted_string = ""
    for segment in timing_data.get("segments", []):
        start = segment.get("start")
        end = segment.get("end")
        text = segment.get("text")
        formatted_string += f"  - 时间: {start:.3f} - {end:.3f}, 文本: \"{text}\"\n"
    return formatted_string.strip()

def align_and_translate(formatted_timings, accurate_text):
    """
    使用OpenAI API来对齐和翻译文本。
    """
    system_prompt = """
    你是一个专业的字幕制作和翻译专家。你的任务是根据两份日文转录本，完成一个精确的、带时间戳的中文翻译。
    一份是“带时间戳的不精确转录本”，它提供了准确的时间信息但文本内容可能有误。
    另一份是“精确转录本”，它提供了准确的文本内容但没有时间信息。

    你的工作流程如下：
    1.  **对齐**：以“带时间戳的不精确转录本”为基础框架，将“精确转录本”中的文本内容，智能地填充到对应的时间段（segment）中。注意，两者的句子和词语不一定完全匹配，但是大部分是match的，尤其是一个segment里的第一个假名和最后一个假名，这是你匹配的重要参考，千万不能错配，你需要根据语义和上下文进行最佳的对齐，确保最终的文本流畅且符合逻辑，最重要的是，精细文本与粗文本的时间戳是一致的。
    2.  **翻译**：将对齐好的、精确的日文文本内容，逐段翻译成流畅、自然的简体中文。
    3.  **输出**：必须以一个JSON数组的格式返回结果，不包含任何额外的解释。每个JSON对象包含以下字段：'start', 'end', 'original_text' (对齐后的精确日文), 'translated_text' (翻译后的中文)。
    
    备注：
    有时精确转录本中可能出现一些语气词在粗转录本中没有的情况，或者相反，这种情况下，忽略这些语气词，只对齐粗转录本中存在的内容。
    """

    user_prompt = f"""
    请处理以下数据：

    --- 带时间戳的不精确转录本 ---
    {formatted_timings}
    ---------------------------------

    --- 精确转录本 ---
    {accurate_text}
    -------------------

    请严格按照指示，完成对齐和翻译，并仅返回JSON格式的输出。
    """
    
    print("正在调用OpenAI API进行处理，请稍候...")
    
    try:
        response = client.chat.completions.create(
            # 推荐使用 gpt-4-turbo 或 gpt-4o
            model="gpt-4.1", 
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            # 开启JSON模式，确保返回格式正确
            response_format={"type": "json_object"} 
        )
        
        # API返回的内容在 response.choices[0].message.content 中
        # 它应该是一个JSON格式的字符串
        response_content = response.choices[0].message.content
        
        # 解析这个JSON字符串
        # GPT返回的JSON可能在外层有一个key，我们需要找到那个包含数组的key
        response_json = json.loads(response_content)
        
        # 假设返回的JSON结构是 {"subtitles": [...]} 或直接是 [...]
        # 我们需要找到那个列表
        for key, value in response_json.items():
            if isinstance(value, list):
                return value
        
        # 如果模型直接返回了一个列表（虽然不太可能在JSON模式下）
        if isinstance(response_json, list):
            return response_json

        print("错误：无法在API响应中找到字幕列表。")
        return None

    except Exception as e:
        print(f"调用OpenAI API时发生错误: {e}")
        return None

def to_srt(data):
    """将处理后的数据转换为SRT字幕格式的字符串"""
    srt_content = ""
    for i, item in enumerate(data, 1):
        start_time = item['start']
        end_time = item['end']
        
        # 格式化时间戳，例如 00:01:02,345
        def format_time(s):
            hours, remainder = divmod(s, 3600)
            minutes, seconds = divmod(remainder, 60)
            milliseconds = int((seconds - int(seconds)) * 1000)
            return f"{int(hours):02}:{int(minutes):02}:{int(seconds):02},{milliseconds:03}"

        start_srt = format_time(start_time)
        end_srt = format_time(end_time)
        
        # 你可以选择只放中文，或者中日双语
        original = item['original_text']
        translated = item['translated_text']
        
        srt_content += f"{i}\n"
        srt_content += f"{start_srt} --> {end_srt}\n"
        srt_content += f"{translated}\n" # 只显示中文
        # srt_content += f"{translated}\n{original}\n" # 如果需要双语
        srt_content += "\n"
        
    return srt_content

def main():
    # --- 文件路径 ---
    json_file_path = Path(f'timestamps/{wav_path.stem}/vocals.json')  
    accurate_text_file_path = f"transcripts/{wav_path.stem}.txt" 
    output_srt_path = f'srts/{wav_path.stem}.srt'

    # 1. 加载数据
    timing_data, accurate_text = load_data(json_file_path, accurate_text_file_path)
    if not timing_data or not accurate_text:
        return

    # 2. 格式化数据以用于Prompt
    formatted_timings = format_timing_data_for_prompt(timing_data)

    # 3. 调用API进行对齐和翻译
    aligned_data = align_and_translate(formatted_timings, accurate_text)
    
    if aligned_data:
        print("\n--- API返回的对齐和翻译结果 ---")
        print(json.dumps(aligned_data, indent=2, ensure_ascii=False))
        
        # 4. 转换为SRT格式
        srt_output = to_srt(aligned_data)
        
        # 5. 保存到文件
        with open(output_srt_path, 'w', encoding='utf-8') as f:
            f.write(srt_output)
            
        print(f"\n成功！SRT字幕文件已保存到: {output_srt_path}")

if __name__ == '__main__':
    main()